<a href="https://colab.research.google.com/github/vikram0050/Real-Time-Facial-Expression-recognition./blob/main/Vikramaditya_Sah_Real_Time_Facial_Expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Introduction
The Indian education landscape has been undergoing rapid changes for the past 10 years owing to the advancement of web-based learning services, specifically, eLearning platforms. Global E-learning is estimated to witness an 8X over the next 5 years to reach USD 2B in 2021. India is expected to grow with a CAGR of 44% crossing the 10M users mark in 2021. Although the market is growing on a rapid scale, there are major challenges associated with digital learning when compared with brick and mortar classrooms. One of many challenges is how to ensure quality learning for students. Digital platforms might overpower physical classrooms in terms of content quality but when it comes to understanding whether students are able to grasp the content in a live class scenario is yet an open-end challenge. In a physical classroom during a lecturing teacher can see the faces and assess the emotion of the class and tune their lecture accordingly, whether he is going fast or slow. He can identify students who need special attention. Digital classrooms are conducted via video telephony software program (exZoom) where it’s not possible for medium scale class (25-50) to see all students and access the mood. Because of this drawback, students are not focusing on content due to lack of surveillance. While digital platforms have limitations in terms of physical surveillance but it comes with the power of data and machines which can work for you. It provides data in the form of video, audio, and texts which can be analysed using deep learning algorithms. Deep learning backed system not only solves the surveillance issue, but it also removes the human bias from the system, and all information is no longer in the teacher’s brain rather translated in numbers that can be analysed and tracked.

**Problem Statements**


We will solve the above-mentioned challenge by applying deep learning algorithms to live video data. The solution to this problem is by recognizing facial emotions.

**Data Description**

The data comes from the past Kaggle challenge "Challenges in Representation Learning: Facial Expression Recognition Challenge":

https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The dataset contains approximately 36K images. we have 7 categories.

train
    angry
    disgust
    fear
    happy
    neutral
    sad
    surprise

validation
    angry
    disgust
    fear
    happy
    neutral
    sad
    surprise
80% of our images are contained inside the train folder, and the last 20% are inside the validation folder.

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing all the required Libraries

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential                                                            
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization,GlobalMaxPool2D,Activation,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
import numpy as np 
import pandas as pd
import cv2
import os
from sklearn.metrics import confusion_matrix
import itertools
print("Tensorflow version:", tf.__version__)

#Data exploration

In [ ]:
# Defining train and validation path
train_path = '/content/drive/MyDrive/Real-Time-Facial-Expression-recognition/train'
val_path = '/content/drive/MyDrive/Real-Time-Facial-Expression-recognition/validation'


In [ ]:
# data categories
categories = os.listdir(train_path)
print(categories)

In [ ]:
# Training images
total_train_images = 0
for dir_ in os.listdir(train_path):
    count = 0
    for f in os.listdir(train_path+'/' + dir_+"/"):
        count += 1
        total_train_images += 1
    print(f"{dir_} has {count} number of images")
    
print(f"\ntotal train images are {total_train_images}")

In [ ]:
# Validation Images
total_validation_images = 0
for dir_ in os.listdir(val_path):
    count = 0
    for f in os.listdir(val_path+'/' + dir_+"/"):
        count += 1
        total_validation_images += 1
    print(f"{dir_} has {count} number of images")
    
print(f"\ntotal validation images are {total_validation_images}")

In [ ]:
# Creating a function for using for show some images from each categories
def imageshow(category):
  plt.figure(figsize= (8,8))
  for i in range(1, 10, 1):
      plt.subplot(3,3,i)
      img = load_img(train_path+'/'+category+"/"+
                    os.listdir(train_path + "/" + category)[i], target_size=(48,48))
      plt.imshow(img)
  plt.suptitle(category,fontsize=30)   
  plt.show()

In [ ]:
#Showing some images from category neutral
imageshow('neutral')

In [ ]:
#Showing some images from category angry
imageshow('angry')

In [ ]:
#Showing some images from category sad
imageshow('sad')

In [ ]:
#Showing some images from category surprise
imageshow('surprise')

In [ ]:
#Showing some images from category happy
imageshow('happy')

In [ ]:
#Showing some images from category disgust
imageshow('disgust')

In [ ]:
#Showing some images from category fear
imageshow('fear')

In [ ]:
# validation set images
for category in categories:
    plt.figure(figsize= (8,8))
    for j in range(1,10,1):
        
        plt.subplot(3,3,j)
        
        img = load_img(val_path+'/'+category+"/"+
                    os.listdir(val_path + "/" + category)[j], target_size=(48,48))
        plt.imshow(img)
    plt.suptitle(category,fontsize=30)
    plt.show()


In [ ]:
# i am resizing image size because vgg16 model is trained on 224,224,3 images
img_size = 224
#Taking batch size 
batch_size = 32

# selecting colour mode as rgb as transfer learning is trained on rgb photos and we have grascaled images
datagen_train = ImageDataGenerator(rescale=1./255)   

# train generators
train_generator = datagen_train.flow_from_directory(train_path,
                                                  target_size=(img_size,img_size),
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

datagen_validation = ImageDataGenerator(rescale=1./255)

# validation generators
validation_generator = datagen_validation.flow_from_directory(val_path,
                                                  target_size=(img_size,img_size),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

In [ ]:
from keras.applications.vgg16 import VGG16
# creating a base model using VGG16 and loading the pretrained weights
vgg = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [ ]:
# making all the layers except last 4 layers non trainable 
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(7, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
# compiling the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
#model summary
model.summary()

In [ ]:
# save model details in png format
utils.plot_model(model, to_file='emotion-model_vgg16.png', show_shapes=True, show_layer_names=True)

In [ ]:
epochs=50
steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch

In [ ]:
# this decreases the learning rate if the model loss does not decrease 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=10, min_lr=0.00001, mode='auto')
# to save model weights 
checkpoint = ModelCheckpoint("vgg16_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
# to early stop
early_stopping=EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=10,
                            verbose=1,
                            restore_best_weights=True)
callbacks = [checkpoint, reduce_lr,early_stopping]

In [ ]:
# Fitting the model 
history = model.fit(
    x=train_generator,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = validation_generator,
    callbacks=callbacks,
    verbose=1)

In [ ]:
model.save('model.h5')

In [ ]:
#loading the model
model_emo = load_model('/content/drive/MyDrive/Real-Time-Facial-Expression-recognition/model (1).h5')